In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')

import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import pandas as pd

In [2]:
import sys
sys.path.append('../src')

from utils import get_immune_genes, process_adata, get_markers

immune_genes = get_immune_genes(mouse=True)
adata = sc.read_h5ad('../data/slideseq/day3_1.h5ad')
adata = process_adata(
    adata, 
    n_top_genes=2000, min_counts=200,
    include_genes=immune_genes, mouse=True)
adata.shape

/ihome/djishnu/alw399/.local/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:282: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number
/ihome/djishnu/alw399/.local/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:569: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


(12998, 2350)

In [3]:
adata.obs['rctd_cluster'].value_counts()

rctd_cluster
0    8550
2    1880
1    1342
4     360
3     343
5     309
6     214
Name: count, dtype: int64

In [4]:
clusters = list(adata.obs['rctd_cluster'].value_counts().keys())
clusters

[0, 2, 1, 4, 3, 5, 6]

In [5]:
import copy
adata_copy = adata.copy()
coex_dir = '../data/slideseq/coexpression/'

In [6]:
adata

AnnData object with n_obs × n_vars = 12998 × 2350
    obs: 'cluster', 'rctd_cluster', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cluster_colors', 'rctd_cluster_colors', 'log1p', 'hvg'
    obsm: 'X_spatial', 'rctd_results', 'spatial'
    layers: 'raw_count'

In [9]:
grnboost = pd.read_csv(coex_dir + 'grnboost2_c0.tsv', sep='\t', names=['source', 'target', 'score'])
grnboost

,source,target,score
0,Cd63-ps,Cd63,1.797708e+01
1,Cd63,Cd63-ps,1.343786e+01
2,Ighm,Jchain,1.145192e+01
3,Hbb-bs,Hbb-bt,9.246247e+00
4,Jchain,Ighm,9.239755e+00
...,...,...,...
314237,Rnf2,Aktip,3.827637e-17
314238,Nsd3,Csnk1a1,3.559937e-17
314239,Bin1,Fabp4,3.271370e-17
314240,Tnip1,Prim1,2.955157e-17


In [ ]:
from arboreto.algo import grnboost2
from distributed import LocalCluster, Client

# Initialize local Dask cluster with 1 worker and 1 thread
local_cluster = LocalCluster(n_workers=1, threads_per_worker=1)
custom_client = Client(local_cluster)

for cluster in clusters:
    print(f'Starting cluster {cluster}')
    
    # Subset adata based on the cluster
    cluster_adata = adata_copy[adata_copy.obs['rctd_cluster'] == cluster]

    # Convert the expression matrix to a DataFrame
    ex_matrix = cluster_adata.to_df()

    # Use gene names from the cluster-specific adata
    tf_names = list(cluster_adata.var_names)

    # Run GRNBoost2 to infer gene regulatory networks
    network = grnboost2(expression_data=ex_matrix,
                        tf_names=tf_names,
                        client_or_address=custom_client,
                        verbose=True)

    # Save the inferred network to a CSV file
    network.to_csv(f'../data/slideseq/coexpression/grnboost2_c{cluster}.tsv', sep='\t', index=False, header=False)


In [6]:
from spaceoracle.tools.GENIE3 import GENIE3, get_link_list

for cluster in clusters:
    cluster_adata = adata_copy[adata_copy.obs['rctd_cluster'] == cluster]
    gene_names = list(cluster_adata.var_names)

    VIM = GENIE3(
        cluster_adata.to_df().values,
        gene_names = gene_names,
        nthreads=8
        )
    
    np.save(f'{coex_dir}c{cluster}', VIM)
    print(f'Finished cluster {cluster}')

    filename = coex_dir + f'links_c{cluster}.txt'
    get_link_list(VIM, gene_names=gene_names, filename=filename)
    

Tree method: RF
K: sqrt
Number of trees: 1000


running jobs on 8 threads


In [ ]:
# links_dict: [source, target, coef_mean, coef_abs, p, -logp]
# coef_matrix_per_cluster: gene x gene coex